Brief guide to run Vietnamese SBERT.

[![Gimme the CODE !](https://img.shields.io/badge/Give%20me-the%20CODE-1abc9c.svg)](https://github.com/Datami555/sentence-transformers)

# Install dependencies

In [ ]:
%pip install -U --no-cache-dir gdown --pre

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
  Created wheel for gdown: filename=gdown-4.4.0-py3-none-any.whl size=14774 sha256=0c36e6fd080e659930b0265a75020e938053dbdc2056676e11acd59e3f0dac91
  Stored in directory: /tmp/pip-ephem-wheel-cache-v_fbppnk/wheels/fb/c3/0e/c4d8ff8bfcb0461afff199471449f642179b74968c15b7a69c
Successfully built gdown
  Attempting uninstall: gdown
    Found existing installation: gdown 4.2.2
    Uninstalling gdown-4.2.2:
      Successfully uninstalled gdown-4.2.2


In [ ]:
!gdown https://drive.google.com/a/gm.uit.edu.vn/uc?id=1pXJZ9eHp6DWkQ5MhCzmWYsKyLQEDiodz&export=download
!tar xzf /content/vn_sbert_deploy.tar.gz

Downloading...
From: https://drive.google.com/a/gm.uit.edu.vn/uc?id=1pXJZ9eHp6DWkQ5MhCzmWYsKyLQEDiodz
To: /content/vn_sbert_deploy.tar.gz
100% 485M/485M [00:04<00:00, 119MB/s]


In [ ]:
!pip -q install transformers
!pip -q install vncorenlp
!pip -q install fairseq
!pip -q install fastBPE

!pip -q install fastapi
!pip -q install uvicorn
!pip -q install pyngrok
!pip -q install pyyaml==5.4.1 

     |████████████████████████████████| 636 kB 15.8 MB/s 


## Download word segmenter

In [ ]:
!mkdir -p vncorenlp/models/wordsegmenter
!wget -q --show-progress https://raw.githubusercontent.com/vncorenlp/VnCoreNLP/master/VnCoreNLP-1.1.1.jar
!wget -q --show-progress https://raw.githubusercontent.com/vncorenlp/VnCoreNLP/master/models/wordsegmenter/vi-vocab
!wget -q --show-progress https://raw.githubusercontent.com/vncorenlp/VnCoreNLP/master/models/wordsegmenter/wordsegmenter.rdr
!mv VnCoreNLP-1.1.1.jar vncorenlp/ 
!mv vi-vocab vncorenlp/models/wordsegmenter/
!mv wordsegmenter.rdr vncorenlp/models/wordsegmenter/

VnCoreNLP-1.1.1.jar 100%[===================>]  26.14M  --.-KB/s    in 0.09s   
vi-vocab            100%[===================>] 514.20K  --.-KB/s    in 0.02s   
wordsegmenter.rdr   100%[===================>] 125.50K  --.-KB/s    in 0.008s  


## Install Vietnamese SBERT

In [ ]:
!git clone https://github.com/DatCanCode/sentence-transformers
!cd sentence-transformers; pip install -e .

Cloning into 'sentence-transformers'...
remote: Enumerating objects: 1200, done.
remote: Counting objects: 100% (181/181), done.
remote: Compressing objects: 100% (22/22), done.
remote: Total 1200 (delta 159), reused 159 (delta 159), pack-reused 1019
Receiving objects: 100% (1200/1200), 30.67 MiB | 16.32 MiB/s, done.
Resolving deltas: 100% (820/820), done.
Obtaining file:///content/sentence-transformers
  Running setup.py develop for sentence-transformers


# Start web server
In-notebook demo is in the next section

In [ ]:
%cd vn_sbert_deploy
!USE_NGROK=True uvicorn server:app

[Errno 2] No such file or directory: 'vn_sbert_deploy'
/content/vn_sbert_deploy
/usr/local/lib/python3.7/dist-packages/distributed/config.py:20: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  defaults = yaml.load(f)
Loading codes from /content/vn_sbert_deploy/bpe/bpe.codes ...
Read 64000 codes from the codes file.
INFO:     Started server process [512]
INFO:uvicorn.error:Started server process [512]
INFO:     Waiting for application startup.
INFO:uvicorn.error:Waiting for application startup.
INFO:     Application startup complete.
INFO:uvicorn.error:Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:8000 (Press CTRL+C to quit)
INFO:uvicorn.error:Uvicorn running on http://127.0.0.1:8000 (Press CTRL+C to quit)
INFO:     2402:800:63b6:bf1c:1132:2e35:3129:55bc:0 - "GET / HTTP/1.1" 200 OK
INFO:     2402:800:63b6:bf1c:1132:2e35:3129:55bc:0 - "GET /fav

# Demo in notebook

Import and load model

In [ ]:
%cd /content/sentence-transformers/
from sentence_transformers import SentenceTransformer
import numpy as np
model = SentenceTransformer('/content/vn_sbert_deploy/phobert_base_mean_tokens_NLI_STS')

/content/sentence-transformers


## Get sentence embeddings

In [ ]:
corpus = ['Cô giáo đang ăn kem.',
          'Cô giáo đang ăn bánh mì.',
          'Chị gái đang thử món thịt dê.',
          'Một anh trai đang cưỡi ngựa.',
          'Kị binh đang đi tuần.',
          'Một vụ tai nạn thảm khốc vừa xảy ra.',
          'Đã có ít nhất hai người chết trong vụ xe khách rơi xuống vực.',
          'Một con khỉ đang diễn xiếc.',
          'Một người đàn ông trong bộ đồ tinh tinh đang làm trò.',
          'Lũ quét đột ngột gây thiệt hại cho ít nhất 3 tỉnh miền núi']

corpus_embeddings = model.encode(corpus)
np.array(corpus_embeddings).shape

(10, 768)

## Semantic clustering

In [ ]:
from sklearn.cluster import KMeans

num_clusters = 5

clustering_model = KMeans(n_clusters=num_clusters)
clustering_model.fit(corpus_embeddings)
cluster_assignment = clustering_model.labels_
cluster_assignment

array([1, 1, 1, 3, 3, 0, 0, 2, 2, 4], dtype=int32)

Applying PCA to reduce the number of dimensions.

In [ ]:
from sklearn.decomposition import PCA
import numpy as np
import pandas as pd


X = np.array(corpus_embeddings)

pca = PCA(n_components=3)
result = pca.fit_transform(X)

In [ ]:
# create dataframe to feed to 

df = pd.DataFrame({
    'sent': corpus,
    'cluster': cluster_assignment.astype(str),
    'x': result[:, 0],
    'y': result[:, 1],
    'z': result[:, 2]
})
df

,sent,cluster,x,y,z
0,Cô giáo đang ăn kem.,1,-5.453616,-2.787267,-3.952195
1,Cô giáo đang ăn bánh mì.,1,-6.913651,-1.272944,-3.702491
2,Chị gái đang thử món thịt dê.,1,-4.696048,-2.625852,1.816635
3,Một anh trai đang cưỡi ngựa.,3,-0.663591,1.092364,6.405024
4,Kị binh đang đi tuần.,3,-1.195645,-0.422664,7.280567
5,Một vụ tai nạn thảm khốc vừa xảy ra.,0,1.948655,5.991241,-1.515969
6,Đã có ít nhất hai người chết trong vụ xe khách...,0,0.658742,5.679992,-2.767126
7,Một con khỉ đang diễn xiếc.,2,5.951980,-5.048024,-1.123377
8,Một người đàn ông trong bộ đồ tinh tinh đang l...,2,7.195945,-4.855567,-1.606090
9,Lũ quét đột ngột gây thiệt hại cho ít nhất 3 t...,4,3.167230,4.248715,-0.834982


In [ ]:
import plotly.express as px


fig = px.scatter_3d(df, x='x', y='y', z='z',
              color='cluster', hover_name='sent',
              range_x = [df.x.min()-1, df.x.max()+1],
              range_y = [df.y.min()-1, df.y.max()+1],
              range_z = [df.z.min()-1, df.z.max()+1])

fig.update_traces(hovertemplate= '<b>%{hovertext}</b>')
fig.show()

/usr/local/lib/python3.7/dist-packages/distributed/config.py:20: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  defaults = yaml.load(f)


# Webserver files
Webserver files in first sections

In [ ]:
#@title model.py
#@markdown Create model.py
model_py="""
from typing import Tuple, List

from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from sklearn.metrics.pairwise import pairwise_distances

import numpy as np
import pandas as pd

import plotly.express as px
import re

from sentence_transformers import SentenceTransformer


class Model(object):
  def __init__(self, model_path):
    super(Model, self).__init__()
    
    self.sbert = SentenceTransformer(model_path)

  def measure_distance(self, sents: Tuple[str, str]):
    # compute embeddings
    corpus_embeddings = self.sbert.encode(sents)

    # compute distance
    distances = (
        pairwise_distances(
        corpus_embeddings[0].reshape(1, -1),
        corpus_embeddings[1].reshape(1, -1),
        metric)[0][0] for metric in ["cosine", "manhattan", "euclidean"]
        )
    return distances

  def fit_kmeans(self, corpus: List[str], n_clusters: int):
    # compute embeddings
    corpus_embeddings = self.sbert.encode(corpus)

    # cluster
    clustering_model = KMeans(n_clusters)
    clustering_model.fit(corpus_embeddings)
    
    # perform PCA
    n_components = int(len(corpus) > 2) + 2
    pca = PCA(n_components)
    X = np.array(corpus_embeddings)
    X_reduced = pca.fit_transform(X)

    # plot corpus in 3d scatter plot
    df = pd.DataFrame({
      'sent': corpus,
      'cluster': clustering_model.labels_.astype(str),
      'x': X_reduced[:, 0],
      'y': X_reduced[:, 1],
      'z': X_reduced[:, 2] if X_reduced.shape[1] > 2 else np.zeros(X_reduced.shape[0])
    })

    fig = px.scatter_3d(df, x='x', y='y', z='z',
              color='cluster', hover_name='sent',
              range_x = [df.x.min()-1, df.x.max()+1],
              range_y = [df.y.min()-1, df.y.max()+1],
              range_z = [df.z.min()-1, df.z.max()+1])

    fig.update_traces(hovertemplate= '<b>%{hovertext}</b>')

    # convert graph to html and replace its id
    graph = fig.to_html(full_html=False, include_plotlyjs=False)

    re_graph = r"Plotly\.newPlot\(\s*'(.*?)',.*?\)"
    groups_html = re.search(re_graph, graph, re.DOTALL)
    result = groups_html[0].replace(groups_html[1], 'plotly')
    
    return result

model = Model('/content/phobert_base_mean_tokens_NLI_STS')

def get_model():
  return model
"""

with open('model.py', 'w') as f:
  f.write(model_py)

In [ ]:
#@title server.py
#@markdown Create server.py

server_py = """
import os
import sys
import traceback

from typing import Tuple, List
from fastapi import FastAPI, Depends, HTTPException, Request
from fastapi.logger import logger
from fastapi.templating import Jinja2Templates
from fastapi.middleware.cors import CORSMiddleware
from pydantic import BaseSettings, BaseModel

from model import Model, get_model

class Settings(BaseSettings):
    BASE_URL = "http://localhost:8000"
    USE_NGROK = os.environ.get("USE_NGROK", "False") == "True"


settings = Settings()


# Initialize the FastAPI app for a simple web server
app = FastAPI()

app.add_middleware(
    CORSMiddleware,
    allow_origins=["*"],
    allow_credentials=True,
    allow_methods=["*"],
    allow_headers=["*"],
)

templates = Jinja2Templates(directory="")

if settings.USE_NGROK:
  
  # pyngrok should only ever be installed or initialized in a dev environment when this flag is set
  from pyngrok import ngrok

  # Get the dev server port (defaults to 8000 for Uvicorn, can be overridden with `--port`
  # when starting the server
  port = sys.argv[sys.argv.index("--port") + 1] if "--port" in sys.argv else 8000

  # Open a ngrok tunnel to the dev server
  public_url = ngrok.connect(port)

  logger.warn("ngrok tunnel \"{}\" -> \"http://127.0.0.1:{}/\"".format(public_url, port))

  # Update any base URLs or webhooks to use the public ngrok URL
  settings.BASE_URL = public_url
  
class DistanceRequest(BaseModel):
  sents: Tuple[str, str]

class DistanceResponse(BaseModel):
  cosine: float
  manhattan: float
  euclidean: float

class KmeansRequest(BaseModel):
  corpus: List[str]
  n_clusters: int

class KmeansResponse(BaseModel):
  plot_html: str
  

@app.get("/")
def root(request: Request):
    return templates.TemplateResponse("index.html", {"request": request, "serveradd": request.url_for('root')})

@app.post("/measure_dis", response_model=DistanceResponse)
def measure_sim(request: DistanceRequest, model:Model = Depends(get_model)):
  try:
    cosine, manhattan, euclidean = model.measure_distance(request.sents)
  except Exception as e:
    raise HTTPException(
          status_code=422,
          detail=traceback.format_exc(),
      )

  return DistanceResponse(
      cosine = cosine,
      manhattan = manhattan,
      euclidean = euclidean
  )

@app.post("/cluster", response_model=KmeansResponse)
def cluster(request: KmeansRequest, model:Model = Depends(get_model)):
  print(request.n_clusters, len(request.corpus))
  if (request.n_clusters < 1):
    raise HTTPException(status_code=400,
                  detail="Number of clusters must be greater than 0.")
  elif (len(request.corpus) < 2):
    raise HTTPException(status_code=400,
                  detail="Corpus must have at least 2 sentences.")
  elif (len(request.corpus) < request.n_clusters):
    raise HTTPException(status_code=400,
                  detail="Number of sentences must be greater than or equal to number of clusters.")
  try:
    result = KmeansResponse(
      plot_html = model.fit_kmeans(request.corpus, request.n_clusters)
    )
  except Exception as e:
    raise HTTPException(
            status_code=422,
            detail=traceback.format_exc(),
          )
  return result
"""

with open('server.py', 'w') as f:
  f.write(server_py)

In [ ]:
#@title index.html
#@markdown Create index.html

index_html = """
<!DOCTYPE html>
<html lang="en">

<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>KLTN</title>

    <link rel="stylesheet" href="https://stackpath.bootstrapcdn.com/bootstrap/4.5.0/css/bootstrap.min.css"
        integrity="sha384-9aIt2nRpC12Uk9gS9baDl411NQApFmC26EwAOH8WgZl5MYYxFfc+NcPb1dKGj7Sk" crossorigin="anonymous">


</head>

<body>

    <nav class="navbar navbar-expand-lg navbar-dark bg-dark">
        <a class="navbar-brand" href="#">
            <img src="https://tuoitre.uit.edu.vn/wp-content/uploads/2015/07/logo-uit-300x248.png" width="40" height="30"
                class="d-inline-block align-top" alt="">
        </a>
        <button class="navbar-toggler" type="button" data-toggle="collapse" data-target="#navbarNav"
            aria-controls="navbarNav" aria-expanded="false" aria-label="Toggle navigation">
            <span class="navbar-toggler-icon"></span>
        </button>
        <div class="collapse navbar-collapse">
            <ul class="navbar-nav">
                <li class="nav-item active">
                    <a class="nav-link" href="#">Playground<span class="sr-only">(current)</span></a>
                </li>
            </ul>
        </div>
    </nav>

    <div class="container-fluid mt-3">
        <div class="row">
            <div class="col-lg">
                <div class="card border-0 rounded">
                    <div class="card-header bg-transparent">
                        <h1>Sentence Embedding for Vietnamese language using Siamese BERT-Network</h1>
                    </div>
                    <div class="card-body">
                        <form class="need-validation">
                            <div class="form-row">
                                <div class="col-sm">
                                    <select class="custom-select" id="task" required>
                                        <option value="">Select a task</option>
                                        <option value="distance">Similarity Measure</option>
                                        <option value="clustering">Clustering</option>
                                    </select>
                                </div>
                                <div class="col-sm">
                                    <div class="input-group" id="numClustersGroup" style="display: none;">
                                        <input type="number" min="1" class="form-control" id="numOfClusters" placeholder="4" aria-describedby="inputGroupPrepend" required>
                                        <div class="input-group-prepend">
                                            <span class="input-group-text" id="inputGroupPrepend">clusters</span>
                                        </div>
                                        <div class="invalid-feedback" id="numOfClusters_feedback"></div>
                                    </div>
                                </div>
                            </div>
                            <div class="form-group mt-2">
                                <label>Enter each sentence on a line.</label>
                                <textarea class="form-control" id="inputText" rows="9" placeholder="Enter 2 sentences to measure similarity or more for clustering."required></textarea>
                                <div class="invalid-feedback" id="inputText_feedback"></div>
                            </div>
                        </form>
                        <div class="row">
                            <div class="col">
                                <button class="btn btn-primary col-sm-4 btn-lg float-right btn-block"
                                    id="doInference">Process</button>
                            </div>
                        </div>
                    </div>
                </div>
            </div>
            <div class="col-lg">
                <div class="card border-0 rounded">
                    <div class="card-header bg-transparent">
                        <h1>Result</h1>
                    </div>
                    <div class="card-body text-center" id=inferenceResult>
                        
                    </div>
                    <div class="card-footer text-center" id="footer"></div>
                </div>
            </div>
        </div>
    </div>

    <script src="https://code.jquery.com/jquery-3.5.1.slim.min.js"
        integrity="sha384-DfXdz2htPH0lsSSs5nCTpuj/zy4C+OGpamoFVy38MVBnE+IbbVYUew+OrCXaRkfj"
        crossorigin="anonymous"></script>
    <script src="https://cdn.jsdelivr.net/npm/popper.js@1.16.0/dist/umd/popper.min.js"
        integrity="sha384-Q6E9RHvbIyZFJoft+2mJbHaEWldlvI9IOYy5n3zV9zzTtmI3UksdQRVvoxMfooAo"
        crossorigin="anonymous"></script>
    <script src="https://stackpath.bootstrapcdn.com/bootstrap/4.5.0/js/bootstrap.min.js"
        integrity="sha384-OgVRvuATP1z7JjHLkuOU7Xw704+h835Lr+6QL9UvYjZE3Ipu6Tp75j7Bh/kR0JKI"
        crossorigin="anonymous"></script>
    <script src="https://bernii.github.io/gauge.js/dist/gauge.min.js"></script>
    <script src="https://cdn.plot.ly/plotly-latest.min.js"></script>
    <script src="https://unpkg.com/axios/dist/axios.min.js"></script>
    <script>
        (() => {
            window.addEventListener('load', () => {
                if ($("#task").val() == "clustering") $("#numClustersGroup").show()
                // Fetch all the forms we want to apply custom Bootstrap validation styles to
                var forms = document.getElementsByClassName('need-validation');
                // Loop over them and prevent submission
                var validation = Array.prototype.filter.call(forms, (form) => {
                form.addEventListener('submit', (event) => {
                    event.preventDefault();
                    event.stopPropagation();
                }, false);
                });
            }, false);
        })();

        $("#task").change(() => {
            if ($("#task").val() == "clustering") {
                $("#numClustersGroup").show()
            } else {
                $("#numClustersGroup").hide()
            }
        });

        // Gauge
        var opts = {
            angle: 0.0, // The span of the gauge arc
            lineWidth: 0.44, // The line thickness
            radiusScale: 1, // Relative radius
            pointer: {
                length: 0.5, // // Relative to gauge radius
                strokeWidth: 0.035, // The thickness
                color: '#000000' // Fill color
            },
            limitMax: false,     // If false, max value increases automatically if value > maxValue
            limitMin: false,     // If true, the min value of the gauge will be fixed
            generateGradient: true,
            highDpiSupport: true,     // High resolution support
            percentColors: [[0.0, "#FF0000"], [0.50, "#0080FF"], [1.0, "#4C9900"]], // !!!!
            strokeColor: '#E0E0E0',
        };

        // do inference
        var inferenceServer = {{ serveradd }};

        var inferenceBtn = $("#doInference");
        inferenceBtn.on("click", () => {
            var task = $("#task").val();
            var text = $("#inputText").val();
            var num_cluster = parseInt($("#numOfClusters").val());
            
            if (!["distance", "clustering"].includes(task) || num_cluster < 1 || !text) return;

            var ArrOfSentences = text.split(/[\r\n]+/).filter(e => e);
            
            if (task == "distance") {
                if (ArrOfSentences.length != 2) {
                    $("#inputText_feedback").html("Please enter 2 sentences to measure distance.")
                    $("#inputText").addClass("is-invalid");
                    return;
                }
                $("#inputText").removeClass("is-invalid");

                // change status of process button to loading
                inferenceBtn.html(
                    '<span class="spinner-border spinner-border-sm" role="status" aria-hidden="true"></span>'
                )
                inferenceBtn.addClass("disabled");

                axios.post(inferenceServer + "/measure_dis", {
                    sents: ArrOfSentences
                })
                .then((response) => {
                    if (!document.getElementById("gauge_sim")) {
                        $("#inferenceResult").html('<canvas id="gauge_sim" height="300" width="500"></canvas>');
                    }
                    cosine_sim = 1 - response.data.cosine;
                    console.log(response.data);
                    $("#footer").html(`<h3>Cosine similarity: ${cosine_sim}</h3>`);

                    var target = document.getElementById('gauge_sim'); // your canvas element
                    var gauge = new Gauge(target).setOptions(opts); // create sexy gauge!
                    gauge.maxValue = 10; // set max gauge value
                    gauge.setMinValue(-10);  // Prefer setter over gauge.minValue = 0
                    gauge.animationSpeed = 15; // set animation speed (32 is default value)
                    gauge.set(cosine_sim * 10); // set actual value
                })
                .catch((error) => console.log(error))
                .finally(() => {
                    // reset status of process btn
                    inferenceBtn.removeClass("disabled");
                    inferenceBtn.html('Process');
                })
            } else {
                if (ArrOfSentences.length < 2) {
                    $("#inputText_feedback").html("Please enter more than 2 sentences to perform clustering.")
                    $("#inputText").addClass("is-invalid");
                    return;
                }
                if (ArrOfSentences.length < num_cluster) {
                    $("#numOfClusters_feedback").html("Number of clusters must be less than or equal to number of sentences.")
                    $("#numOfClusters").addClass("is-invalid");
                    return;
                }

                $("#numOfClusters").removeClass("is-invalid");
                $("#inputText").removeClass("is-invalid");

                window.PLOTLYENV=window.PLOTLYENV || {};

                if (!document.getElementById("plotly")) {
                    $("#inferenceResult").html('<div id="plotly" class="plotly-graph-div" style="height:100%; width:100%;"></div>');
                }
                
                $("#footer").html('');

                // change status of process button to loading
                inferenceBtn.html(
                    '<span class="spinner-border spinner-border-sm" role="status" aria-hidden="true"></span>'
                )
                inferenceBtn.addClass("disabled");

                axios.post(inferenceServer + "/cluster", {
                    corpus: ArrOfSentences,
                    n_clusters: num_cluster
                })
                .then((response) => {
                    eval(response.data.plot_html);
                    console.log(response.data.plot_html);
                })
                .catch((error) => console.log(error))
                .finally(() => {
                    // reset status of process btn
                    inferenceBtn.removeClass("disabled");
                    inferenceBtn.html('Process');
                })

                
            }
        })
    </script>
</body>

</html>
"""

with open("index.html", 'w') as f:
  f.write(index_html)